# Programming Languages and Environments 
## (Lecture 10)

### Syllabus

- Merge sort

In [ ]:
let rec split_n n l = 
  if n = 0 then ([], l)
  else match l with
  | [] -> ([], l)
  | x::xs -> let (l1,l2) = split_n (n-1) xs in (x::l1,l2)

let split l =
  let n = List.length l in
  split_n (n/2) l

In [ ]:
let _ = assert (split [1;2;3;4;5;6] = ([1;2;3], [4;5;6]))
let _ = assert (split_n 0 [1;2;3;4;5;6] = ([], [1;2;3;4;5;6]))
let _ = assert (split_n 1 [1;2;3;4;5;6] = ([1], [2;3;4;5;6]))
let _ = assert (split_n 10 [1;2;3;4;5;6] = ([1;2;3;4;5;6],[]))

In [ ]:
let rec merge l1 l2 = 
  match l1, l2 with
  | [], l -> l
  | l, [] -> l
  | x::xs, y::ys -> if x < y then x::(merge xs l2) else y::(merge l1 ys)  

In [ ]:
let _ = assert (merge [1;3;5] [2;4;6] = [1;2;3;4;5;6])
let _ = assert (merge [1;3;5;7] [2;6] = [1;2;3;5;6;7])
let _ = assert (merge [] [1;3] = [1;3])
let _ = assert (merge [1;3] [] = [1;3])

In [ ]:
let rec mergesort l = 
  match l with 
  | [] | [_] -> l 
  | _ -> let (l1,l2) = split l in merge (mergesort l1) (mergesort l2)

In [ ]:
let _ = assert (mergesort [3; 2; 1; 4; 5; 6; 7; 8; 9; 10] = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10])

In [ ]:
let rec mergesort l =
  let rec split l = match l with
    | [] -> ([], [])
    | [x] -> ([x], [])
    | x::y::tl -> let (l1, l2) = split tl in (x::l1, y::l2)
  in let rec merge l1 l2 = match (l1, l2) with
    | ([], l) -> l
    | (l, []) -> l
    | (x::tl1, y::tl2) -> if x < y then x::(merge tl1 l2) else y::(merge l1 tl2)
  in match l with 
  | [] | [_] -> l 
  | _ -> let (l1,l2) = split l in merge (mergesort l1) (mergesort l2)

In [ ]:
let _ = assert (mergesort [3; 2; 1; 4; 5; 6; 7; 8; 9; 10] = [1; 2; 3; 4; 5; 6; 7; 8; 9; 10])

In [ ]:
let split_n n l =  (* tail recursive *)
  let rec split_n_rec n l acc = 
    if n = 0 then (List.rev acc, l)
    else match l with
    | [] -> (List.rev acc, l)
    | x::xs -> split_n_rec (n-1) xs (x::acc)
  in split_n_rec n l []

In [ ]:
let _ = assert (split_n 0 [1;2;3;4;5;6] = ([], [1;2;3;4;5;6]))
let _ = assert (split_n 1 [1;2;3;4;5;6] = ([1], [2;3;4;5;6]))
let _ = assert (split_n 10 [1;2;3;4;5;6] = ([1;2;3;4;5;6],[]))

In [ ]:
let merge l1 l2 = (* tail recursive *)
  let rec merge_rec l1 l2 acc =
    match l1,l2 with
    | [], l -> (List.rev acc)@l
    | l, [] -> (List.rev acc)@l
    | x::xs, y::ys -> if x < y then merge_rec xs l2 (x::acc) else merge_rec l1 ys (y::acc)
  in merge_rec l1 l2 []

In [ ]:
let _ = assert (merge [1;3;5] [2;4;6] = [1;2;3;4;5;6])
let _ = assert (merge [1;3;5;7] [2;6] = [1;2;3;5;6;7])
let _ = assert (merge [] [1;3] = [1;3])
let _ = assert (merge [1;3] [] = [1;3])

## BST

In [ ]:
type 'a tree = Empty | Node of 'a * 'a tree * 'a tree

In [ ]:
let rec insert x t = 
  match t with
  | Leaf -> Node(x, Leaf, Leaf)
  | Node(y, l, r) -> if x < y then Node(y, insert x l, r) else Node(y, l, insert x r)

In [ ]:
let rec remove_min t =
  match t with
  | Leaf -> failwith "Tree is empty"
  | Node(y, Leaf, r) -> (y, r)
  | Node(y, l, r) -> let (m, l') = remove_min l in (m, Node(y, l', r))
  

In [ ]:
let _ = assert (remove_min (Node(5, Node(3, Leaf, Leaf), Node(7, Leaf, Leaf))) = (3, Node(5, Leaf, Node(7, Leaf, Leaf))))

In [ ]:
let rec remove x t = 
  match t with
  | Leaf -> Leaf
  | Node(y,Leaf,r) when x = y -> r (* maybe superfluous *)
  | Node(y,l,Leaf) when x = y -> l 
  | Node(y,l,r) when x = y -> let (m, r') = remove_min r in Node(m, l, r')
  | Node(y,l,r) -> if x < y then Node(y, remove x l, r) else Node(y, l, remove x r)

In [ ]:
let _ = assert (remove 3 (Node(5, Node(3, Leaf, Leaf), Node(7, Leaf, Leaf))) = Node(5, Leaf, Node(7, Leaf, Leaf)))
let _ = assert (remove 7 (Node(5, Node(3, Leaf, Leaf), Node(7, Leaf, Node(9,Leaf,Leaf)))) = Node(5, Node(3, Leaf, Leaf), Node(9, Leaf, Leaf)))

## N-ary trees

In [ ]:
type 'a ntree = NEmpty | NNode of 'a * 'a ntree list

(* type 'a nntree = NNEmpty | NNNEmpty of 'a netree

type 'a netree = NNode of 'a * 'a netree list *)

let nt = NNode (1, [NNode (2, [NNode (3, [NEmpty])]); NNode (4, []); NNode(5,[])])


In [ ]:
let rec sum_tree t = 
  let rec sum_aux l = 
    match l with 
    | [] -> 0
    | t'::ts -> sum_tree t' + sum_aux ts
  in 
  match t with 
  | NEmpty -> 0
  | NNode(v, l) -> v + sum_aux l 

In [ ]:
let rec sum_tree t = 
  match t with 
  | NEmpty -> 0
  | NNode(v, l) -> List.fold_left (fun acc t' -> acc + sum_tree t') v l 

In [ ]:
let rec sum_tree t = 
  let rec sum_rec l = 
    match l with 
    | [] -> 0 
    | t::ts -> sum_tree t + sum_rec ts
  in 
  match t with 
  | NEmpty -> 0
  | NNode (v,l) -> v + sum_rec l

In [ ]:
let rec sum_tree t = 
  match t with 
  | NEmpty -> 0
  | NNode (v,l) -> v + List.fold_left (fun acc t' -> acc + sum_tree t') 0 l 

In [ ]:
let _ = sum_tree nt

In [ ]:
let rec sum_tree t = 
  match t with
  | NEmpty -> 0
  | NNode (v, l) -> v + List.fold_left (fun acc t -> acc + sum_tree t) 0 l

In [ ]:
let _ = sum_tree nt

In [ ]:
let rec map_ntree f nt = 
  match nt with
  | NEmpty -> NEmpty
  | NNode (v, l) -> NNode (f v, List.map (fun t' -> map_ntree f t') l)

In [ ]:
let rec map_ntree f nt = 
  match nt with
  | NEmpty -> NEmpty
  | NNode (v, l) -> NNode (f v, List.map (map_ntree f) l)

In [ ]:
let _ = map_ntree ((+) 1) nt

In [ ]:
let rec prefix_fold_ntree f acc nt = 
  match nt with
  | NEmpty -> acc
  | NNode (v, l) -> let acc_v = f acc v in List.fold_left (fun acc nt' -> prefix_fold_ntree f acc nt') acc_v l

In [ ]:
let rec prefix_fold_ntree f acc nt = 
  match nt with
  | NEmpty -> acc
  | NNode (v, l) -> let acc_v = f acc v in List.fold_left (prefix_fold_ntree f) acc_v l

In [ ]:
let rec prefix_fold_ntree f acc nt = 
  let rec fold_aux acc l = 
    match l with 
    | [] -> acc
    | nt'::nts -> let acc_nt' = prefix_fold_ntree f acc nt' in fold_aux acc_nt' nts 
  in 
  match nt with
  | NEmpty -> acc
  | NNode (v, l) -> let acc_v = f acc v in fold_aux acc_v l          

In [ ]:
let rec prefix_fold_ntree f acc nt =
  match nt with
  | NEmpty -> acc
  | NNode (v, l) -> let acc_n = f acc v in List.fold_left (fun acc t -> prefix_fold_ntree f acc t) acc_n l

In [ ]:
let rec prefix_fold_ntree f acc nt =
  match nt with
  | NEmpty -> acc
  | NNode (v, l) -> let acc_n = f acc v in List.fold_left (prefix_fold_ntree f) acc_n l

In [ ]:
let _ = prefix_fold_ntree (+) 0 nt

In [ ]:
let _ = prefix_fold_ntree (fun acc x -> acc@[x]) [] nt

In [ ]:
let _ = assert (List.rev (prefix_fold_ntree (fun acc x -> x::acc) [] nt) = [1;2;3;4;5])

In [ ]:
let _ = prefix_fold_ntree (fun bst x -> insert x bst) Leaf nt

In [ ]:
let _ = nt 
let _ = prefix_fold_ntree (fun (acc:'a bst) x -> insert x acc) Leaf nt
let _ = assert (prefix_fold_ntree (fun (acc:'a bst) x -> insert x acc) Leaf nt = Node (1, Leaf,
 Node (2, Leaf, Node (3, Leaf, Node (4, Leaf, Node (5, Leaf, Leaf))))))

let _ = List.fold_left (fun (acc:'a bst) x -> insert x acc) Leaf [3;1;4;0;8;3;5]